## How to use a Large Language Models (LLMs) to analyse text and create novel datasets.

### - Load text from PDFs

Last updated: 2024-05-18

See this blog post on Autonomous Econ for a deatiled wlkthrough of this notebook: https://autonomousecon.substack.com/p/how-anyone-can-use-ai-to-create-novel

### Install dependencies

In [1]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


In [2]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [3]:
pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.6 MB/s eta 0:00:00


In [5]:
pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.2 MB/s eta 0:00:00


In [19]:
pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 6.3 MB/s eta 0:00:00


### Import libraries

In [25]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
import os
import json
import pandas as pd
from collections import defaultdict

### Set enviornment variables and model

In [8]:
# Set OpenAI API key
os.environ["OPENAI_API_KEY"] = "<YOUR_API_KEY>"

In [95]:
# set model and model params
model="gpt-4"
chat = ChatOpenAI(temperature=0.0, model_name=model)

See https://platform.openai.com/docs/models for a full list of models to choose from.

### Choose the web links to load into your document list

In [21]:
loader_pdf = PyPDFLoader("mps-snaphsot-feb-24.pdf")
docs = loader_pdf.load()



In [120]:
loader = DirectoryLoader('mps_snapshots/', glob="**/*.pdf", loader_cls=PyPDFLoader)

docs=loader.load()

In [121]:
# Number of pages of content
len(docs)

12

In [122]:
# Group documents by their metadata source and merge their content
content_by_source = defaultdict(str)
for doc in docs:
    source = doc.metadata['source']
    content_by_source[source] += doc.page_content  # Assuming `doc.page_content` contains the text of the page

In [123]:
# Number of OCR anouncements / sources
len(content_by_source)

4

Create the prompt template

In [124]:

# create the instructions that will convert the LLM response into a JSON format

date_schema = ResponseSchema(name="ocr_date", description="date of OCR announcement")

decision_schema = ResponseSchema(
    name="rate_decision", description="decision for the OCR"
)

ocr_rate_schema = ResponseSchema(
    name="ocr", description="OCR decision by the Monetary Policy Committee"
)

policy_stance_schema = ResponseSchema(
    name="policy_stance", description="policy stance of statement ranging from -1 (very dovish) to 1 (very hawkish)"
)


response_schemas = [
    date_schema,
    ocr_rate_schema,
    decision_schema,
    policy_stance_schema,
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"ocr_date": string  // date of OCR announcement
	"ocr": string  // OCR decision by the Monetary Policy Committee
	"rate_decision": string  // decision for the OCR
	"policy_stance": string  // policy stance of statement ranging from -1 (very dovish) to 1 (very hawkish)
}
```


In [125]:
# Create the extraction instructions for the prompt template

extract_template = """\
For the following text, extract the following information:

date_schema: date of the OCR announcement. report the date in the following format: mm/YYYY and as a pandas timestamp.

decision_schema: decision by the Monetary Policy Committee for the OCR. Only calssify as either 'raise','hold' or 'lower'.

ocr_rate_schema: OCR decision by the Monetary Policy Committee. Report the OCR as a float to 2 decimal places.

policy_stance_schema: classify the text with the following values depending on how dovish or hawkish the overall message was in the OCR announcement.
-1: Strongly expresses a belief that the economy may be growing too slowly and/or inflation is too low and may need stimulus through monetary policy.
-0.5: Overall message expresses a belief that the economy may be growing too slowly and/or inflation is too low and may need stimulus through monetary policy.
0: Expresses neither a hawkish nor dovish view and the Fed is on track to achieve its employment and inflation goals.
0.5: Overall message expresses a belief that the economy is growing too quickly and/or inflation is too high and may need to be slowed down through monetary policy.
1: Strongly expresses a belief that the economy is growing too quickly and/or inflation is too high and may need to be slowed down through monetary policy.

text: {text}

{format_instructions}
"""

In [126]:

prompt = ChatPromptTemplate.from_template(template=extract_template)


### Run your prompt over every document in your document list

In [127]:
# Generate a response for each unique metadata source
output_list = []
for source, content in content_by_source.items():
    messages = prompt.format_messages(text=content, format_instructions=format_instructions)
    response = chat(messages)
    parsed_response = output_parser.parse(response.content)
    output_list.append(parsed_response)

In [128]:

# JSON list
output_list


[{'ocr_date': '05/2022',
  'ocr': '2.0',
  'rate_decision': 'raise',
  'policy_stance': '1'},
 {'ocr_date': '11/2023',
  'ocr': '5.50',
  'rate_decision': 'hold',
  'policy_stance': '0.5'},
 {'ocr_date': '02/2024',
  'ocr': '5.50',
  'rate_decision': 'hold',
  'policy_stance': '0.5'},
 {'ocr_date': '05/2020',
  'ocr': '0.25',
  'rate_decision': 'hold',
  'policy_stance': '-1'}]

In [135]:

# Convert your json list into a pandas DataFrame
json_list = json.dumps(output_list)
df = pd.read_json(json_list)


In [136]:
# Convert 'ocr_date' to pandas datetime
df['ocr_date'] = pd.to_datetime(df['ocr_date'], format='%m/%Y')

# Sort the DataFrame by the 'ocr_date' column in descedning order
df = df.sort_values(by='ocr_date',ascending=False)

# Convert to 'mm/YYYY' format and make it a string for display purposes
df['ocr_date'] = df['ocr_date'].dt.strftime('%m/%Y')

In [119]:
df

,ocr_date,ocr,rate_decision,policy_stance
3,2024-02-01,5.50,hold,0.5
1,2023-11-01,5.50,hold,0.5
2,2023-08-01,5.50,hold,0.5
0,2022-05-01,2.00,raise,1.0
4,2020-05-01,0.25,hold,-1.0


In [137]:
df

,ocr_date,ocr,rate_decision,policy_stance
2,02/2024,5.50,hold,0.5
1,11/2023,5.50,hold,0.5
0,05/2022,2.00,raise,1.0
3,05/2020,0.25,hold,-1.0


In [ ]:
# save the structured df to a csv file
df.to_csv("ocr_sample.csv", index=False)